# Model Evaluation by playing them against one another!

In [1]:
%%capture
%load_ext autoreload
%autoreload 2

from lib.arena import RandoTron, Battle, Player
import numpy as np

In [2]:
player1 = RandoTron(name = 'RandoTron v1')
player2 = RandoTron(name = 'RandoTron vAlpha')

In [3]:
battle = Battle(player1, player2, verbose = 1)
np.random.seed(42)
battle.play_match(10)

148/148 [==============================] - 1s 7ms/step
10 games were played between RandoTron v1 and RandoTron vAlpha with 0 draws.
Result was a statistically improbable tie!
Player 1 on average won in a game of length 84.0.
Player 2 on average won in a game of length 67.2
Overall average length of game was 75.6
Total time taken: 6.5s at
 - 653ms per finished game.
 - 9ms per move in a finished game


In [4]:
battle = Battle(player1, player2, verbose = 0)
np.random.seed(42)
battle.play_match(100)

100 games were played between RandoTron v1 and RandoTron vAlpha with 0 draws.
The winner was RandoTron v1 with a 53.0% win rate!
Player 1 on average won in a game of length 77.7.
Player 2 on average won in a game of length 78.7
Overall average length of game was 78.15
Total time taken: 1:11 at
 - 716ms per finished game.
 - 9ms per move in a finished game
